In [1]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob
from PIL import Image
from datetime import datetime


def analyze_sentiment(text: str) -> str:
    """Classify text sentiment via TextBlob polarity."""
    score = TextBlob(text).sentiment.polarity
    if score > 0.1:
        return "Positive"
    elif score < -0.1:
        return "Negative"
    else:
        return "Neutral"


def blank_image(width: int = 400, height: int = 300,
                color: tuple[int, int, int] = (255, 255, 255)) -> Image.Image:
    """Return a plain white PIL.Image (placeholder)."""
    return Image.new("RGB", (width, height), color)


def generate_dashboard(file, start_date: str, end_date: str):
    # 1. Load CSV safely
    try:
        df = pd.read_csv(file.name)
    except Exception:
        return pd.DataFrame(), plt.figure(), *(blank_image(),) * 3

    if {"date", "feedback"} - set(df.columns):
        return pd.DataFrame(), plt.figure(), *(blank_image(),) * 3

    # 2. Pre-process
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])
    df["feedback"] = df["feedback"].astype(str)
    df["sentiment"] = df["feedback"].apply(analyze_sentiment)

    # Date-range filter
    try:
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date)
    except Exception:
        return pd.DataFrame(), plt.figure(), *(blank_image(),) * 3

    df = df.loc[(df["date"] >= start) & (df["date"] <= end)]
    if df.empty:
        return pd.DataFrame(), plt.figure(), *(blank_image(),) * 3

    # 3. Sentiment-over-time plot
    daily_sentiment = (
        df.groupby(["date", "sentiment"]).size().unstack().fillna(0)
    )
    fig, ax = plt.subplots()
    daily_sentiment.plot(ax=ax)
    ax.set_title("Sentiment Over Time")
    ax.set_ylabel("Count")
    ax.set_xlabel("Date")
    ax.legend(title="Sentiment")
    plt.xticks(rotation=45)
    fig.tight_layout()

    # 4. Word clouds
    wc_images: dict[str, Image.Image] = {}
    for sentiment in ["Positive", "Neutral", "Negative"]:
        text = " ".join(df.loc[df["sentiment"] == sentiment, "feedback"])
        if text.strip():
            wc = WordCloud(width=400, height=300, background_color="white")
            img = wc.generate(text).to_image()         # PIL.Image
            wc_images[sentiment] = img
        else:
            wc_images[sentiment] = blank_image()

    # 5. Return outputs (order matches Interface.outputs)
    return (
        df,
        fig,
        wc_images["Positive"],
        wc_images["Neutral"],
        wc_images["Negative"],
    )

demo = gr.Interface(
    fn=generate_dashboard,
    inputs=[
        gr.File(label="Upload CSV with 'date' and 'feedback' columns"),
        gr.Textbox(label="Start Date (YYYY-MM-DD)", value="2025-05-01"),
        gr.Textbox(label="End Date (YYYY-MM-DD)", value="2025-07-10"),
    ],
    outputs=[
        gr.Dataframe(label="📄 Filtered Data"),
        gr.Plot(label="📈 Sentiment Over Time"),
        gr.Image(label="☁️ Positive Word Cloud"),
        gr.Image(label="☁️ Neutral Word Cloud"),
        gr.Image(label="☁️ Negative Word Cloud"),
    ],
    title="📊 Sentiment Dashboard for Customer Feedback",
    description=(
        "Upload a CSV with **date** and **feedback** columns. "
        "Select a date range to analyze sentiment trends and view word-clouds."
    ),
)

if __name__ == "__main__":
    demo.launch()


c:\Users\V16RKhalil\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
